In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import re
import regex
from pprint import pprint

import sys
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

import csv 

In [2]:
from pdf_processing import *

TODO: 
 - write documentation
 - locations not included after the date (might need to be taken care of manually)
 - footnotes (like in People_of_Greece.pdf and Senate_Floor_Immigration_Reform.pdf)



DONE: 
 - Q&As / interviews
     - function to count the number of occurrences of "Obama:", "Question:", etc.
   
      
 - locations with multiple comma's (ex. when Washington, D.C. is included) --> might be better to store location as one elements instead of splitting it up, the location can be cleaned up afterwards
     - location is not split up anymore, but kept as a single element (will be easier to clean up afterwards)

In [3]:
# pdfs that have these in the title might have to be excluded: interview, presser, Press_Conference, press, meeting, 
# debate, hall (maybe)
multiple_people_talking = ['Barack_Obama_-_Al-Arabiya_Interview.pdf', 'CGI_2013.pdf', 
                           'Midterm_Elections_Presser_2014.pdf', 'YSEALI_Fellows.pdf', 
                           'PM_Abe_of_Japan_Joint_Presser.pdf', 'Paris_Press_Conference_2015.pdf',
                           'Gun_Violence_Presser.pdf', 'Associated_Press_Luncheon.pdf', 'Final_Press_Conference.pdf',
                           'CNN_Guns_in_America_Town_Hall.pdf', 'START_Treaty_Presser.pdf', 
                           'Midterm_Elections_Presser_2010.pdf', 'Joint_Presser_with_President_Benigno_Aquino.pdf',
                           'Worldwide_Troop_Talk.pdf', 'Post_ASEAN_Presser_2016.pdf', 'Post_G7_Presser_Japan.pdf',
                           'Presser_Unannounced_Heath_Care.pdf', 'Kenya_Civil_Society_Meeting.pdf',
                           'Obama-Romney_-_Second_Live_Debate.pdf', 'Jamaica_Town_Hall.pdf', 'YSEALI_Town_Hall.pdf',
                           'BP_Oil_Spill_Presser.pdf', 'VA_Misconduct_Allegations.pdf', 'ISIL_Pentagon_Update.pdf',
                           'ISIL_Presser_Post_Security_Council_Meeting.pdf']
# maybe: 'Alabama_Tornado_Devestation.pdf', 'NAACP_Coference_2015.pdf'

# the year for Community_College_Plan.pdf is 20105 but should be 2015
# Recovery_and_Reinvestment_Act_2016.pdf has the year missing from the date

Get the path to the directory in which the PDFs are stored.

In [4]:
pdf_dir = Path.cwd().parent / "pdfs"

Filepaths of all the PDFs in the folder `pdf_dir`, can be used to iterate over all the PDFs to store the extracted speeches in a dataframe.

In [5]:
pdfs = list(pdf_dir.glob('*.pdf'))  
print("current number of PDFs:", len(pdfs))

current number of PDFs: 436


Get filepath of the PDF you want to process.

In [6]:
filepath = pdfs[355]
print("File:", str(filepath.parts[-1]))

File: Chemical_Weapons_Syria.pdf


Make a PDFHandler object for the given filepath.

In [7]:
pdf = PDFHandler(filepath)

Print the last page of the PDF before it has been processed.

In [8]:
print(pdf.original_page(-1))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
But we are the United States of America, and we cannot and must not turn a blind eye to 
what happened in Damascus. Out of the ashes of world war, we built an international order 
and enforced the rules that gave it meaning. And we did so because we believe that the rights 
of individuals to live in peace and dignity depends on the responsibilities of nations. We aren't 
 
perfect, but this nation more than any other has been willing to meet those responsibilities.
So to all members of Congress of both parties, I ask you to take this vote for our national 
security. I am looking forward to the debate. And in doing so, I ask you, members of 
Congress, to consider that some things are more important than partisan differences or the 
 
politics of the moment. 
Ultimately, this is not about who occupies this office at any given time; it's about who we are 
as a country. I believe that the people's representatives must be invested in what A

Print the first page of the PDF before it has been processed.

In [9]:
print(pdf.original_page(0))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
Barack Obama 
On The Use of Chemical Weapons by Syrian Government 
 
Delivered 31 August 2013, White House, Washington, D.C.
 
 
AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio 
Ten days ago, the world watched in horror as men, women and children were massacred in 
Syria in the worst chemical weapons attack of the 21st century. Yesterday the United States 
presented a powerful case that the Syrian government was responsible for this attack on its 
 
own people.
Our intelligence shows the Assad regime and its forces preparing to use chemical weapons, 
launching rockets in the highly populated suburbs of Damascus, and acknowledging that a 
chemical weapons attack took place. And all of this corroborates what the world can plainly 
see -- hospitals overflowing with victims; terrible images of the dead. All told, well over 1,000 
people were murdered. Several hundred of them were children -- young girls and boys 

Define a regular expression to get the date, location, and content of the speech. Extract the entire speech from the PDF.

In [10]:
start = r"^(?:\s*AA *RR\s+mmeerriiccaann *hheettoorriicc\.\.ccoomm|\s*AmericanRhetoric\.com|\s*AAmmeerriiccaannRRhheettoorriicc\.\.ccoomm)?"

datemarker =r"\s*(delivered|First *Broadcast|Uploaded[ a-z]*|Published)?"
date = r"(((?P<day>[0-9]{1,2}) *(?P<mon>[a-z]{1,10}))|((?P<mon>[a-z]{1,10}) *(?P<day>[0-9]{1,2}))),? +(?P<year>[0-9]* *)"
loc = r"(([,.] *location *)?[,.] *(?P<location>.+?))?"
auth = r"(?:.?AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio( *and edited for continuity)?.?)?"

content = r"(?P<content>.*)"

#footnotes = r"^(([1-9][0-9]*.+?)+\n+)?"
tr = r"(Transcription *by *[a-z.\- ]+\s*)?"
url = r"((Property *of *)?AmericanRhetoric\.com)?"
page = r"(page *[0-9]+\s*$)"

full_pat = start + \
r"(\s+.*?" + r"^(" + datemarker + " *" + date + r")" + loc + r"\s+" + auth + r")?" + \
r"\s+" + content + \
r"^(?:\s*" + tr + url + r".*?" + page + r")"
pat = regex.compile(full_pat, regex.I | regex.DOTALL | regex.M | regex.V1)

speech = pdf.extract_speech(pat)
print(speech)

Ten days ago, the world watched in horror as men, women and children were massacred in 
Syria in the worst chemical weapons attack of the 21st century. Yesterday the United States 
presented a powerful case that the Syrian government was responsible for this attack on its 
 
own people.
Our intelligence shows the Assad regime and its forces preparing to use chemical weapons, 
launching rockets in the highly populated suburbs of Damascus, and acknowledging that a 
chemical weapons attack took place. And all of this corroborates what the world can plainly 
see -- hospitals overflowing with victims; terrible images of the dead. All told, well over 1,000 
people were murdered. Several hundred of them were children -- young girls and boys gassed 
 
to death by their own government.
This attack is an assault on human dignity. It also presents a serious danger to our national 
security. It risks making a mockery of the global prohibition on the use of chemical weapons. 
It endangers our frien

Print the relevant info of the PDF.

In [11]:
pdf.print_info()

Title: Chemical_Weapons_Syria
Number of pages: 4
Date: 31 August 2013
Location: White House, Washington, D.C.


Check whether there are multiple speakers. (If there is a high "Obama:" count, this might indicate he is answering questions, i.e., it is an interview, press conference, debate,...)

In [12]:
count = ('obama:', 'president:', 'question:', 'audience:', 'member:')
counts, speakers = pdf.multiple_speakers(speech, count)
print(counts)
print()
pprint(speakers)

{'obama:': 0, 'president:': 0, 'question:': 0, 'audience:': 0, 'member:': 0}

['decision:', 'community:', 'people:']


Replace substrings.

In [13]:
rep_old = [r'"', r'’’', r'‘‘', r'“', r'”', r'[sic]']
rep_new = [r'' , r''  , r''  , r'' , r'' , r' '    ]

clean_speech = pdf.substring_replace(speech, rep_old, rep_new)

Replace or delete some characters to clean the speech (only the ones that need to be matched with a regular expression).

In [14]:
re_old = [r'\s+', r'\[(Source:)?\s*http.*?\]', r'[\[\]\(\)]']
re_new = [r' '  , r''                        , r''          ]

clean_speech = pdf.match_replace(clean_speech, re_old, re_new)
print(clean_speech)

Ten days ago, the world watched in horror as men, women and children were massacred in Syria in the worst chemical weapons attack of the 21st century. Yesterday the United States presented a powerful case that the Syrian government was responsible for this attack on its own people. Our intelligence shows the Assad regime and its forces preparing to use chemical weapons, launching rockets in the highly populated suburbs of Damascus, and acknowledging that a chemical weapons attack took place. And all of this corroborates what the world can plainly see -- hospitals overflowing with victims; terrible images of the dead. All told, well over 1,000 people were murdered. Several hundred of them were children -- young girls and boys gassed to death by their own government. This attack is an assault on human dignity. It also presents a serious danger to our national security. It risks making a mockery of the global prohibition on the use of chemical weapons. It endangers our friends and our par

Output to csv file.

In [15]:
csv_filename = Path.cwd().parent / "speeches_csv" / "speeches_436.txt"

fields = ["title", "pages", "date", "location", "highest_speaker_count", "content"]

# use 'a+' instead of 'w' if you want to append at the bottom of the file instead of replacing the file
with open(csv_filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL) 
    
    # for i in range(len(pdfs)):      
    # 0,10 - 10,20 - 20,40 - 40,60 - 60,80 - 80,100 - 100,120 - 120,140 - 140,160 - 160,180 - 180,200 - 200,220 - 
    # 220,240 - 240,260 - 260,280 - 280,300 - 300,320 - 320,340 - 340,360 - 360,380 - 380,400 - 400,420 - 420,436
    for i in range(420,436):  
        if i==0:
            csvwriter.writerow(fields)
            
        filepath = pdfs[i]
        pdf = PDFHandler(filepath)
        row = np.array(pdf.full_extract(pat, count, rep_old, rep_new, re_old, re_new))
        csvwriter.writerow(row)
        
        print("pdf", i)
        print('\n')
        print(pdf.original_page(0))
        print('\n' + 50*'-' + '\n')
        print(row[:-1])
        print('\n')
        print(row[-1][:500])
        print('\n' + 50*'-' + '\n')
        print(row[-1][-500:])
        print('\n' + 100*'-' + '\n\n')

Merge the separate csv files.

In [16]:
csv_dir = Path.cwd().parent / "speeches_csv" 
csvs = sorted(list(csv_dir.glob('*.txt')))

path_merged = csv_dir / "all_speeches.txt"

for path in csvs:
    tmp = pd.read_csv(path)
    tmp.to_csv(path_merged, index=False, header=True, mode='a+')